# Transportation

$$
\begin{align}
    \text{min} \quad & \sum_{i \in I}\sum_{j \in J} c_{i, j} x_{i, j} \\
    \text{s.t.} \quad & \sum_{j \in J} x_{i, j} \leq b_{i} & \forall \; i \in I \\
    \quad & \sum_{i \in I} x_{i, j} = d_{j} & \forall \; j \in J \\
    & x_{i, j} \geq 0 & \forall \;i \in I, j \in J \\
\end{align}
$$

In [17]:
# Python native modules
import json

# Third-party packages
import pandas as pd  # Tabular data
import pyomo.environ as pyo

In [18]:
# Read input file and store in local variable `input_data`
with open("input_transp.json", mode="r", encoding="utf8") as file:
    input_data = json.load(file)

In [19]:
# Create local variables to store input parameters
availabilities = input_data["availabilities"]
demands = input_data["demands"]

# Dictionary of costs indexed by tuples (origin, destination)
costs = {
    (c["from"], c["to"]): c["value"]
    for c in input_data["costs"]
}

In [20]:
# Create model instance
# model = ###

In [21]:
# Sets for suppliers I and customers J
# model.I = pyo.Set(initialize=###)
# model.J = pyo.Set(initialize=###)

In [22]:
# Paramters
# model.b = pyo.Param(###, initialize=###)
# model.d = pyo.Param(###, initialize=###)
# model.c = pyo.Param(###, ###, initialize=###)

In [23]:
# Decision variables
# model.x = pyo.Var(###, ###, within=###)

In [24]:
# Supplier availablity constraints
# def av_cstr(###, ###):
#     return ### <= ###


# model.av_cstr = pyo.Constraint(###, rule=###)

In [25]:
# Demand equality constraints
# def dem_cstr(###, ###):
#     return ### == ###


# model.dem_cstr = pyo.Constraint(###, rule=###)

In [26]:
# Objective function
# def obj(###):
#     total_cost = sum(
#         ### * ###
#         for i in model.I
#         for j in model.J
#     )
#     return total_cost


# model.obj = pyo.Objective(rule=###, sense=###)

In [27]:
# Instantiate Highs persistent solver (make sure highspy is installed)
solver = pyo.SolverFactory("appsi_highs")

In [28]:
# Apply method solve
solver.solve(model)

{'Problem': [{'Lower bound': 526.0, 'Upper bound': 526.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 1}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [29]:
# Use objective as a callable to see its value
model.obj()

526.0

In [31]:
sol = [
    {"from": i, "to": j, "value": val}
    for (i, j), val in model.x.extract_values().items()
]

In [35]:
dataframe = pd.DataFrame(sol).pivot(
    index="from", columns="to", values="value"
)

In [36]:
display(dataframe)

to,C1,C2,C3,C4
from,,,,
S1,0.0,2.0,0.0,12.0
S2,0.0,11.0,15.0,0.0
S3,5.0,0.0,0.0,5.0
